# Design Docs for New CMC

### Current Problems

- No test code
- No formatter, linter
- Too many arguments
- Only few combinations of the arguments works OK
- T is one dimensional while X is two
- Examples are not good
- Dual SGD is not supported
- Kernel mismatch between QB and KCMC
- Unstable continuous treatment model
- No consistency plot
- No doubly robust
- No information criterion
- No cross-validation
- No hypothesis testing

### Plans: Create Multiple Classes
- BasePolicy
- BaseCMCEstimator
  - fit
  - predict
- KCMCEstimator
  - attributes: problem, constraints, if there is
  - Primal, CVXPY, Hessian, Information Criterion, Hypothesis Testing (asymptotics, confint)
- DRKCMCEstimator?
- GPKCMCEstimator
  - fit_predict only
- DualKCMCEstimator
  - fit (SGD), use Nystrom with KPCA
- DualNCMCEstimator
  - fit (SGD)
- k_fold_cv, 
- test_*.py
- type it with mypy
- add docstring
- formatter
- ~maybe use jax for Hessian and future use~ We only need an average of Hessian, which is Hessian of the average

### Where to reduce the redundancy?
- choice of kernel (use the QB scaling)
- no intersection of uncertainty set
- no Hajek constraints
- provide true p_t (with a good example) or estimate it independently
- no normalize_p_t (expricitly provide normalized one if necessary)
- no testing for policy (test on the difference of policy value or doubly-robust method is more interesting)
- only assume R or [K] as the action space

### What to do?
- Read DR-paper
  - See how important DR is
  - Understand how they apply asymptotics
  - DR might reduce the variance in trace calculation

In [6]:
import numpy as np
import cvxpy as cp

In [7]:
# Generate a random non-trivial quadratic program.
m = 15
n = 10
p = 5
np.random.seed(1)
P = np.random.randn(n, n)
P = P.T @ P
q = np.random.randn(n)
G = np.random.randn(m, n)
h = G @ np.random.randn(n)
A = np.random.randn(p, n)
b = np.random.randn(p)

# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, P) + q.T @ x),
                 [G @ x <= h,
                  A @ x == b])
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution corresponding to the inequality constraints is")
print(prob.constraints[0].dual_value)


The optimal value is 86.891415855699
A solution x is
[-1.68244521  0.29769913 -2.38772183 -2.79986015  1.18270433 -0.20911897
 -4.50993526  3.76683701 -0.45770675 -3.78589638]
A dual solution corresponding to the inequality constraints is
[ 0.          0.          0.          0.          0.         10.45538054
  0.          0.          0.         39.67365045  0.          0.
  0.         20.79927156  6.54115873]


In [9]:
ineq, eq = prob.constraints

In [11]:
ineq.dual_value

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       10.45538054,  0.        ,  0.        ,  0.        , 39.67365045,
        0.        ,  0.        ,  0.        , 20.79927156,  6.54115873])